In [2]:
import env_methods as em
import os
import datetime as dt
import pickle
import math

In [3]:
file_path = r'C:\Users\bpara\Geo data\GloFAS total\Full period data'

In [4]:
with open(file_path, 'rb') as f:
        miss_data = pickle.load(f)

In [4]:
#This is a function that, when given a dict with a time datakey containing datetime objects after 1900 (non-inclusive),
#returns a list of tuples containing all of the start and end indices (inclusive, noninclusive) of wateryears in the time interval

def get_wyr_indices(data):
    result = []
    
    for today in range(len(data["time"])):
        if data["time"][today].month == 10 and data["time"][today].day == 1:
            #check for leap years
            if (data["time"][today].year + 1) % 4 == 0:
                index = em.find_closest_val(data["time"][today] + dt.timedelta(days = 366), data["time"])
                if data["time"][index].month == 10 and data["time"][index].day == 1:
                    result.append((today, index))
            else:
                index = em.find_closest_val(data["time"][today] + dt.timedelta(days = 365), data["time"])
                if data["time"][index].month == 10 and data["time"][index].day == 1:
                    result.append((today, index))
    
    return result

In [5]:
#This is a function that, when given a dict with lat, lon, time and datakey keys, finds the maximum value of the datakey value of a certain (valid) lat lon pair in every complete water year.

def get_wyr_max(data, datakey, lat, lon):
    lat_i = em.find_closest_val(lat, data["lat"])
    lon_i = em.find_closest_val(lon, data["lon"])
    
    result = []
    
    for timepair in get_wyr_indices(data):
        maxval = data[datakey][timepair[0]][lat_i][lon_i]
        maxday = data["time"][timepair[0]]
        for day in range(len(data[datakey][timepair[0]:timepair[1]])):
            if data[datakey][timepair[0]:timepair[1]][day][lat_i][lon_i] > maxval:
                maxval = data[datakey][timepair[0]:timepair[1]][day][lat_i][lon_i]
                maxday = data["time"][timepair[0]:timepair[1]][day]
        result.append((maxval, maxday))
    
    result.sort(key = lambda x: -x[0])
    
    return result

In [6]:
#This is a function that, when given a dict with lat, lon, time and datakey keys, returns a dict with lat, lon and maxima keys. 
#The maxima key returns a 2-d array where dict["maxima"][lat][lon] is the ordered list of maximum values for each water year at that lat lon


#User beware, this takes a while to run
def get_wyr_maxima(data, datakey):
    result = dict()
    result["lon"] = data["lon"]
    result["lat"] = data["lat"]
    result["maxima"] = []
    
    for lat in data["lat"]:
        print("starting on latitude " + str(lat))
        row = []
        for lon in data["lon"]:
            if math.floor(lon + 0.1) != math.floor(lon):
                print("starting on longitude " + str(lon))
            row.append(get_wyr_max(data, datakey, lat, lon))
            
        result["maxima"].append(row)
    
    return result

In [7]:
maxima_result = get_wyr_maxima(miss_data, "dis24")

file = open("Maxima result", "wb")
pickle.dump(maxima_result, file)
file.close()

starting on latitude 50.35
starting on longitude -114.04999999999998
starting on longitude -113.04999999999998
starting on longitude -112.04999999999998
starting on longitude -111.04999999999998
starting on longitude -110.04999999999998
starting on longitude -109.04999999999998
starting on longitude -108.04999999999998
starting on longitude -107.04999999999998
starting on longitude -106.04999999999998
starting on longitude -105.04999999999998
starting on longitude -104.04999999999998
starting on longitude -103.04999999999998
starting on longitude -102.04999999999998
starting on longitude -101.04999999999998
starting on longitude -100.04999999999998
starting on longitude -99.04999999999998
starting on longitude -98.04999999999998
starting on longitude -97.04999999999998
starting on longitude -96.04999999999998
starting on longitude -95.04999999999998
starting on longitude -94.04999999999998
starting on longitude -93.04999999999998
starting on longitude -92.04999999999998
starting on lon

In [19]:
get_wyr_max(miss_data, "dis24", 33, -89)

[(60.10274, datetime.datetime(1990, 1, 27, 0, 0)),
 (42.81778, datetime.datetime(2019, 2, 24, 0, 0)),
 (35.19556, datetime.datetime(2001, 3, 6, 0, 0)),
 (26.440445, datetime.datetime(1987, 3, 3, 0, 0)),
 (26.227364, datetime.datetime(2011, 9, 7, 0, 0)),
 (23.66551, datetime.datetime(1980, 4, 1, 0, 0)),
 (19.949455, datetime.datetime(2002, 9, 28, 0, 0)),
 (17.653795, datetime.datetime(1983, 4, 9, 0, 0)),
 (16.558979, datetime.datetime(2009, 3, 28, 0, 0)),
 (15.698406, datetime.datetime(1998, 1, 9, 0, 0)),
 (13.46002, datetime.datetime(2004, 2, 7, 0, 0)),
 (13.360752, datetime.datetime(2005, 8, 31, 0, 0)),
 (13.266192, datetime.datetime(1989, 5, 7, 0, 0)),
 (13.004855, datetime.datetime(1991, 5, 28, 0, 0)),
 (12.948164, datetime.datetime(2012, 12, 18, 0, 0)),
 (12.436475, datetime.datetime(2010, 2, 7, 0, 0)),
 (11.848819, datetime.datetime(1993, 5, 27, 0, 0)),
 (11.620502, datetime.datetime(2014, 4, 16, 0, 0)),
 (11.353495, datetime.datetime(2006, 10, 21, 0, 0)),
 (10.627938, datetime.da

In [25]:
lat_i = em.find_closest_val(38.65, maxima_result["lat"])
lon_i = em.find_closest_val(-90.15, maxima_result["lon"])

maxima_result["maxima"][lat_i][lon_i]



[(22097.963, datetime.datetime(1993, 7, 27, 0, 0)),
 (17395.594, datetime.datetime(2019, 5, 31, 0, 0)),
 (17260.055, datetime.datetime(1986, 10, 6, 0, 0)),
 (13703.196, datetime.datetime(1995, 5, 20, 0, 0)),
 (12662.667, datetime.datetime(1998, 7, 3, 0, 0)),
 (12586.428, datetime.datetime(2008, 6, 15, 0, 0)),
 (12451.319, datetime.datetime(1984, 6, 26, 0, 0)),
 (11989.559, datetime.datetime(1990, 6, 22, 0, 0)),
 (11428.803, datetime.datetime(1999, 5, 21, 0, 0)),
 (11181.37, datetime.datetime(1993, 10, 1, 0, 0)),
 (10962.005, datetime.datetime(1986, 7, 15, 0, 0)),
 (10938.56, datetime.datetime(2017, 5, 1, 0, 0)),
 (10173.924, datetime.datetime(2001, 6, 8, 0, 0)),
 (9831.311, datetime.datetime(1983, 5, 3, 0, 0)),
 (9552.103, datetime.datetime(1996, 6, 10, 0, 0)),
 (9337.51, datetime.datetime(2018, 9, 9, 0, 0)),
 (9272.054, datetime.datetime(2014, 7, 5, 0, 0)),
 (9258.963, datetime.datetime(2010, 7, 1, 0, 0)),
 (8799.824, datetime.datetime(2002, 5, 15, 0, 0)),
 (8262.836, datetime.datetim

In [24]:
lon_i

248